In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt
from idaes.apps.uncertainty_propagation.sens import get_dsdp

    'pyomo.common.sorting.sorted_robust'.  Please update your import.
    (deprecated in 6.1) (called from <frozen importlib._bootstrap>:228)


## Define fixed bed model 

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y):
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}}
    return dv_dict_overall

In [9]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
initialize_point = './priors/20210916_feed313_bath293.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#initialize_point = './priors/20210916_feed313_bath293_5e3.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [10]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=303.15, y=0.2, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [14]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
#dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}
dv_pass = {'temp_feed': None, 'yfeed': None}

measurement = ['FCO2', 'temp']
measurement_extra = {'FCO2': [19], 'temp': [19]}

t_measure = exp_time_


In [15]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


## Compute FIM

In [ ]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
##sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(313.15, 293.15, 0.15)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output='store_output_fco2',
                                formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.design_values)

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
print(doe_object.dsdp)

In [ ]:
import pickle

with open('store_output_botht', 'rb') as f:
    data = pickle.load(f)
    f.close()
print(data)

In [ ]:
doe_object.models[0]

In [ ]:
breakthrough_modify2(doe_object.models[2])

In [ ]:
def breakthrough_modify2(m, file=None, source="lab"):
    '''
    Draw breakthrough curve of the Pyomo model
    
    Arguments:
        m: Pyomo model
        file: when source = 'computer', this is where the computer experimental data is stored. Otherwise, its default is None
        source: if computer, it is plotting the computer experiment data. If lab, its comparing the experiments from the lab
    
    Return: None 
    
    Other: plot
    
    '''
    
    
    # unit: mol/m3
    outlet_den, _, _, _ = extract3d(m, m.C, 'CO2')
    
    # unit: cm/s
    outlet_vel, _, _, _ = extract2d(m, m.v)
    
    outlet_n2, _, _, _ = extract3d(m, m.C, 'N2')
    
    model_temp, _,_,_ = extract2d(m,m.temp)
    
    
    T = []
    for j in m.t:
        T.append(value(j))
        t_final = T[-1]
        
    
    if source == "lab":
        break_wvu = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/breakthrough_wvu.csv')
    
        exp = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/co2_breakthrough.csv')
        
        data_c1 = exp['FCO2']
        data_c = np.asarray(data_c1)
        data_t1 = exp['time']  
        data_t = np.asarray(data_t1*60+10)
        new_data = np.interp(T, data_t, data_c)
    
        fco2 = []
        for t in m.t:
            fco2.append(value(m.FCO2[0,19,t]))
        
        yco2 = []
        for i in range(len(fco2)):
            yco2.append(fco2[i]/fco2[-1])
            
        # real value of FCO2. for verification
        comp = outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1])
    
        plt.plot(exp['time']*60, exp['FCO2'],'b.', color='r', label='Experimental data')
        #plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.plot(T, yco2, label='Model prediction')
        
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
    elif source == "computer": 
        
        sol = pd.read_csv(file)
    
        #for i in range(len(time_exp)):    
        #    yco2_exp[time_exp[i]] = sol['FCO2'][i]
        #    temp_mid_exp[time_exp[i]] = sol['temp_mid'][i]
        #    temp_end_exp[time_exp[i]] = sol['temp_end'][i]
        
        exp_fco2 = np.asarray(sol['FCO2'].values.tolist())
        exp_temp_mid = np.asarray(sol['temp_mid'].values.tolist())
        exp_temp_end = np.asarray(sol['temp_end'].values.tolist())
        
        plt.plot(T, exp_fco2/exp_fco2[-1], label = 'Experimental data')
        plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, exp_temp_mid, label='Experimental data of middle T')
        plt.plot(T, model_temp[10,:], label='Model prediction of middle T')
        plt.plot(T, exp_temp_end, label='Experimental data of end T')
        plt.plot(T, model_temp[19,:], label='Model prediction of end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.title('Temperature model prediction and experimental data')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()

## Gradient-based optimization 

In [16]:
# feed, bath, y
exp1 = generate_exp(313.15, 293.15, 0.15)



In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=None)

Sensitivity information is scaled by its corresponding parameter nominal value.
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is ./priors/20210916_feed313_bath293.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  313.15
yfeed is fixed at  0.15
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-ps


Number of Iterations....: 145

                                   (scaled)                 (unscaled)
Objective...............:   6.2598356165211637e+00    6.2598356165211637e+00
Dual infeasibility......:   1.8401383972133090e-06    1.8401383972133090e-06
Constraint violation....:   2.9590072254427613e-12    7.2759576141834259e-11
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   2.9590072254427613e-12    1.8401383972133090e-06


Number of objective function evaluations             = 152
Number of objective gradient evaluations             = 4
Number of equality constraint evaluations            = 152
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 147
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 145
Total CPU secs in IPOPT (w/o function evaluations)   =     67.679
Total CPU secs in NLP function evalua

  44  5.4943313e+00 1.22e+01 2.74e+12  -1.0 2.65e-02  14.4 1.00e+00 1.00e+00H  1
  45  5.4943324e+00 1.58e+01 2.33e+12  -1.0 3.64e-03  14.9 1.00e+00 1.00e+00H  1
  46  5.4943359e+00 1.42e+01 1.69e+12  -1.0 1.07e-02  14.4 1.00e+00 1.00e+00H  1
  47  5.4943374e+00 1.34e+01 1.59e+12  -1.0 2.58e-03  14.8 1.00e+00 1.00e+00H  1
  48  5.4943399e+00 1.29e+01 1.81e+12  -1.0 9.39e-03  14.3 1.00e+00 5.56e-01h  1
  49  5.4943399e+00 1.29e+01 1.72e+12  -1.0 4.37e-05  16.6 4.16e-01 4.16e-01s 21
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50r 5.4943399e+00 1.29e+01 1.00e+03   0.7 0.00e+00  18.8 0.00e+00 0.00e+00R  1
  51r 5.4095340e+00 6.60e+00 3.85e+03   0.7 5.18e+03    -  1.04e-03 9.90e-04f  1
  52  5.4105362e+00 6.60e+00 1.12e+04  -1.0 7.24e+00    -  5.56e-01 2.57e-04h  1
  53  5.4100126e+00 6.60e+00 3.24e+04  -1.0 3.60e+03    -  1.00e-04 1.43e-05h  1
  54  5.4100108e+00 6.60e+00 1.42e+05  -1.0 6.54e+03    -  3.52e-06 1.83e-08h  7
  55  5.4077704e+00 6.60e+00

 137r 1.2589938e+01 2.70e+00 1.00e+03   0.4 0.00e+00  16.7 0.00e+00 4.77e-07R 22
 138r 1.1824065e+01 2.28e+00 9.97e+02   0.4 2.35e+03    -  2.87e-03 1.14e-03f  1
 139  1.2566616e+01 6.22e+00 9.77e+03  -1.0 7.98e+00    -  4.96e-01 2.86e-01H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 140  1.2566616e+01 6.21e+00 7.21e+10  -1.0 4.04e-06  16.3 9.90e-01 1.00e+00h  1
 141  1.2566616e+01 6.20e+00 1.32e+11  -1.0 2.21e-05  15.8 1.00e+00 1.00e+00h  1
 142  1.2566616e+01 6.12e+00 1.74e+11  -1.0 8.82e-05  15.3 1.00e+00 1.00e+00h  1
 143  1.2566614e+01 5.82e+00 2.07e+11  -1.0 3.16e-04  14.8 1.00e+00 1.00e+00h  1
 144  1.2566608e+01 4.79e+00 2.15e+11  -1.0 9.78e-04  14.3 1.00e+00 1.00e+00h  1
 145  1.2566586e+01 1.79e+00 2.29e+11  -1.0 3.11e-03  13.9 1.00e+00 1.00e+00h  1
 146  1.2566510e+01 1.81e+01 2.59e+11  -1.0 1.15e-02  13.4 1.00e+00 1.00e+00h  1
 147  1.2566248e+01 2.34e+01 1.25e+11  -1.0 1.35e-02  12.9 1.00e+00 1.00e+00h  1
 148  1.2566137e+01 9.37e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 230  4.2560776e+00 2.39e+00 2.72e+10  -1.0 1.03e-02  12.4 1.00e+00 1.00e+00h  1
 231  4.2560816e+00 2.39e+00 5.15e+10  -1.0 7.08e-03  12.8 1.00e+00 1.00e+00h  1
 232  4.2560832e+00 2.39e+00 9.59e+10  -1.0 4.98e-03  13.2 1.00e+00 1.00e+00h  1
 233  4.2560839e+00 2.39e+00 1.64e+11  -1.0 3.28e-03  13.7 1.00e+00 1.00e+00h  1
 234  4.2560841e+00 2.39e+00 2.63e+11  -1.0 1.94e-03  14.1 1.00e+00 1.00e+00h  1
 235  4.2560843e+00 2.39e+00 3.42e+11  -1.0 1.03e-03  14.5 1.00e+00 1.00e+00h  1
 236  4.2560846e+00 2.39e+00 7.97e+11  -1.0 7.32e-03  14.0 1.00e+00 1.00e+00h  1
 237  4.2560847e+00 2.39e+00 1.33e+12  -1.0 3.91e-03  14.5 1.00e+00 1.00e+00h  1
 238  4.2560848e+00 2.39e+00 1.66e+12  -1.0 2.13e-03  14.9 1.00e+00 1.00e+00h  1
 239  4.2560850e+00 2.48e+01 4.09e+12  -1.0 1.60e-02  14.4 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 240  4.2560855e+00 1.17e+01

 322  4.2563471e+00 2.86e+01 1.60e+12  -1.0 7.74e-03  14.5 1.00e+00 1.00e+00h  1
 323  4.2563492e+00 9.95e+00 8.96e+11  -1.0 1.00e-02  14.1 1.00e+00 1.00e+00h  1
 324  4.2563554e+00 3.39e+00 3.98e+11  -1.0 1.00e-02  13.6 1.00e+00 1.00e+00h  1
 325  4.2563577e+00 5.25e+00 3.97e+11  -1.0 4.13e-03  14.0 1.00e+00 1.00e+00h  1
 326  4.2563649e+00 1.22e+01 3.38e+11  -1.0 8.64e-03  13.5 1.00e+00 1.00e+00h  1
 327  4.2563676e+00 5.91e+00 3.70e+11  -1.0 6.50e-03  14.0 1.00e+00 1.00e+00h  1
 328  4.2563757e+00 2.47e+00 5.23e+11  -1.0 1.63e-02  13.5 1.00e+00 1.00e+00h  1
 329  4.2563788e+00 5.71e+00 3.55e+11  -1.0 5.89e-03  13.9 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 330  4.2563882e+00 3.09e+00 1.51e+11  -1.0 6.08e-03  13.4 1.00e+00 1.00e+00h  1
 331  4.2564165e+00 3.75e+00 7.63e+10  -1.0 7.64e-03  13.0 1.00e+00 1.00e+00h  1
 332  4.2564272e+00 2.32e+00 1.05e+11  -1.0 4.88e-03  13.4 1.00e+00 1.00e+00h  1
 333  4.2564595e+00 9.24e+00

 418  1.0561879e+01 3.51e+00 9.35e+02  -1.0 5.26e+00    -  3.40e-01 2.00e-01H  1
 419  1.0561855e+01 3.18e+00 1.36e+11  -1.0 1.69e-03  14.0 9.90e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 420  1.0561712e+01 2.41e+00 1.72e+11  -1.0 4.95e-03  13.6 1.00e+00 1.00e+00h  1
 421  1.0561631e+01 2.35e+00 2.39e+11  -1.0 2.46e-03  14.0 1.00e+00 1.00e+00h  1
 422  1.0561308e+01 1.83e+01 8.93e+11  -1.0 1.81e-02  13.5 1.00e+00 1.00e+00h  1
 423  1.0561156e+01 1.62e+01 7.77e+11  -1.0 6.20e-02  13.0 4.92e-01 1.25e-01h  4
 424  1.0560225e+01 1.28e+01 5.94e+11  -1.0 8.85e-03  12.6 1.00e+00 2.50e-01h  3
 425  1.0558761e+01 4.37e+00 1.46e+11  -1.0 7.64e-03  13.0 1.00e+00 1.00e+00h  1
 426  1.0553562e+01 2.19e+00 2.93e+10  -1.0 4.53e-03  12.5 1.00e+00 1.00e+00h  1
 427  1.0535651e+01 2.19e+00 1.45e+10  -1.0 1.60e-02  12.0 1.00e+00 1.00e+00h  1
 428  1.0472177e+01 2.18e+00 7.59e+10  -1.0 3.77e-02  11.5 1.00e+00 1.00e+00H  1
 429  1.0446402e+01 2.17e+00

 511  2.1682090e+01 8.73e+00 1.61e+10  -1.0 4.09e-02  11.7 1.00e+00 5.12e-02h  2
 512  2.3052027e+01 8.71e+00 1.61e+10  -1.0 1.18e-01  11.3 1.00e+00 1.27e-02h  2
 513  2.3318159e+01 8.70e+00 1.63e+10  -1.0 4.65e-02  11.7 1.00e+00 3.07e-02h  4
 514  3.3214478e+01 7.67e+00 2.05e+10  -1.0 1.33e-01  11.2 1.00e+00 9.12e-01h  1
 515  3.3639381e+01 6.17e+00 2.01e+10  -1.0 3.78e-01  10.7 1.00e+00 5.00e-01h  2
 516  3.4797540e+01 6.01e+00 2.39e+10  -1.0 2.11e-02  12.1 1.00e+00 1.00e+00h  1
 517  3.4854742e+01 5.49e+00 2.65e+10  -1.0 7.04e-02  11.6 1.00e+00 1.00e+00h  1
 518  3.5208711e+01 5.28e+00 2.94e+10  -1.0 2.92e-02  12.0 1.00e+00 1.00e+00h  1
 519  3.5462454e+01 5.20e+00 3.24e+10  -1.0 1.21e-02  12.4 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 520  3.5698497e+01 4.93e+00 3.47e+10  -1.0 3.88e-02  12.0 1.00e+00 1.00e+00h  1
 521  3.5906894e+01 4.14e+00 3.46e+10  -1.0 1.17e-01  11.5 1.00e+00 1.00e+00h  1
 522  3.6325822e+01 2.31e+00

 604  2.1464590e+00 3.55e+00 6.89e+07  -1.0 4.08e-01   7.9 3.71e-01 9.62e-01h  1
 605  2.1478087e+00 7.82e-01 3.26e+07  -1.0 9.89e-02   7.5 1.00e+00 1.00e+00f  1
 606  2.1439203e+00 8.07e-02 5.73e+06  -1.0 1.58e-01   7.0 1.00e+00 1.00e+00h  1
 607  2.1436689e+00 4.29e-02 7.83e+06  -1.0 1.47e-02   8.3 1.00e+00 5.00e-01h  2
 608  2.1434120e+00 4.17e-02 7.77e+06  -1.0 6.01e-03   8.7 1.00e+00 1.00e+00h  1
 609  2.1432683e+00 4.92e-03 4.00e+06  -1.0 2.79e-03   9.2 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  2.1427446e+00 5.27e-03 3.47e+06  -1.0 7.26e-03   8.7 1.00e+00 1.00e+00h  1
 611  2.1425254e+00 6.45e-03 2.57e+06  -1.0 2.02e-03   9.1 1.00e+00 1.00e+00h  1
 612  2.1419059e+00 7.88e-03 1.25e+06  -1.0 2.76e-03   8.6 1.00e+00 1.00e+00h  1
 613  2.1407263e+00 7.18e-03 1.10e+06  -1.0 2.92e-03   8.2 1.00e+00 1.00e+00f  1
 614  2.1400822e+00 3.80e-03 2.88e+05  -1.0 6.09e-03   7.7 1.00e+00 1.00e+00f  1
 615  2.1412814e+00 1.54e-02

In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Heatmap

In [ ]:
#design_ranges = [list(np.linspace(293,333,5)), list(np.linspace(0.1,0.3,6))]
design_ranges = [[293, 303, 313, 323, 333], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

dv_apply_name = ['temp_feed','yfeed']

dv_apply_time = [[0],[0]]

# Define experiments
exp1 = generate_exp(313.15, 303.15, 0.15)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

print(design_ranges)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=False,
                                    scale_nominal_param_value=True, 
                                     store_name='20210919_heatmap', read_name='20210919_heatmap'
                                    )



In [ ]:
test = all_fim.extract_criteria()

fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','yfeed'], 'Fixed bed','$feed T [K]$', 'y' )

In [ ]:
na = ['temp_feed', 'yfeed']
dd = ['temp_feed', 'yfeed']
a = [[293.0, 303.0, 313.0, 323.0, 333.0], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

In [ ]:
print(a[na.index('temp_feed')])